<a href="https://colab.research.google.com/github/vitorsr/ccd/blob/master/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Dados de Queimadas na Amazônia

## Bibliotecas Utilizadas

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

O objetivo aqui é analisar a Influência das Queimadas ocorrentes na Floresta Amazônica com variações na temperatura local. Para isto, utilizaremos dois Datasets:


*   O primeiro com Dados de Medições de Temperatura em várias estações do País;
*   O segundo com o resgistro de Queimadas no Brasil.



## Leitura dos Dados

<br><b>variável | descrição | unidade </b></br>
<br>date | data e hora da coleta             -
<br> id | ID da estação de coleta             -
<br>prec | precipitação |       mm
<br>tair | temperatura do ar  | graus Celsius
<br>tw | temperatura de bulbo úmido |  graus Celsius
<br>tmax | temperatura máxima do ar  | graus Celsius
<br>tmin | temperatura mínima do ar  | graus Celsius
<br>urmax | umidade relativa máxima  |             %
<br>patm | pressão atmosférica         |    hPa
<br>pnmm | pressão atmosférica média ao nível do mar  |           hPa
<br>wd | direção do vento        |   graus
<br>wsmax  |  rajadas de vento    |         m/s
<br>n |  horas de sol          |     h
<br>cc |   cobertura de nuvens    |           -
<br>evap  |   evaporação        |      mm
<br>ur  |   umidade relativa          |     %
<br>ws   |   velocidade do vento       |      m/s

In [0]:
df = pd.read_csv("inmetr.csv")
df.head()

,date,id,prec,tair,tw,tmax,tmin,urmax,patm,pnmm,wd,wsmax,n,cc,evap,ur,ws
0,1970-05-04 00:00:00,83010,NaN,NaN,NaN,32.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.75,NaN
1,1970-05-04 12:00:00,83010,NaN,25.3,24.0,NaN,23.7,90.0,1005.9,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
2,1970-05-04 18:00:00,83010,NaN,29.2,27.1,NaN,NaN,85.0,1004.2,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
3,1970-05-05 00:00:00,83010,NaN,25.0,27.0,32.6,NaN,92.0,1007.5,NaN,NaN,NaN,NaN,8.0,2.4,88.25,NaN
4,1970-05-05 12:00:00,83010,0.0,25.9,24.8,NaN,23.0,91.0,1006.4,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN


In [0]:
df_aux = pd.read_csv("bdmep_meta.csv")
df_aux.head()

,id,lon,lat,alt,name,state,uf,time_zone,offset_utc,time_zone.1,offset_utc.1
0,83010,-68.733333,-11.016667,260.00,Brasiléia,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
1,82704,-72.666667,-7.633333,170.00,Cruzeiro do Sul,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
2,82915,-67.800000,-9.966667,160.00,Rio Branco,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
3,82807,-70.766667,-8.166667,190.00,Tarauacá,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
4,83098,-36.166667,-10.150000,56.13,Coruripe,Alagoas,AL,America/Maceio,-3,America/Maceio,-3


O primeiro arquivo contém os dados principais de Medições Metereológicas, enquanto o segundo apenas serve de referência para a Localização das Estações de Medição.

Os dados apresentados abragem o período entre os anos de 1970 e 2018.

Vamos iniciar o processamento realizando um Join entre os dois DataFrames.

In [0]:
temp_data = pd.merge(df,df_aux,on='id')
temp_data.head()

,date,id,prec,tair,tw,tmax,tmin,urmax,patm,pnmm,wd,wsmax,n,cc,evap,ur,ws,lon,lat,alt,name,state,uf,time_zone,offset_utc,time_zone.1,offset_utc.1
0,1970-05-04 00:00:00,83010,NaN,NaN,NaN,32.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.75,NaN,-68.733333,-11.016667,260.0,Brasiléia,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
1,1970-05-04 12:00:00,83010,NaN,25.3,24.0,NaN,23.7,90.0,1005.9,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,-68.733333,-11.016667,260.0,Brasiléia,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
2,1970-05-04 18:00:00,83010,NaN,29.2,27.1,NaN,NaN,85.0,1004.2,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,-68.733333,-11.016667,260.0,Brasiléia,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
3,1970-05-05 00:00:00,83010,NaN,25.0,27.0,32.6,NaN,92.0,1007.5,NaN,NaN,NaN,NaN,8.0,2.4,88.25,NaN,-68.733333,-11.016667,260.0,Brasiléia,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
4,1970-05-05 12:00:00,83010,0.0,25.9,24.8,NaN,23.0,91.0,1006.4,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,-68.733333,-11.016667,260.0,Brasiléia,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5


Retirando colunas sem informações relevantes para esta análise.

In [0]:
temp_data = temp_data.drop(['prec','patm','pnmm','wd','wsmax','n','cc','evap','ur','ws','lat','lon','alt','time_zone','offset_utc','time_zone.1','offset_utc.1','uf'],axis=1)
temp_data.head()

,date,id,tair,tw,tmax,tmin,urmax,name,state
0,1970-05-04 00:00:00,83010,NaN,NaN,32.6,NaN,NaN,Brasiléia,Acre
1,1970-05-04 12:00:00,83010,25.3,24.0,NaN,23.7,90.0,Brasiléia,Acre
2,1970-05-04 18:00:00,83010,29.2,27.1,NaN,NaN,85.0,Brasiléia,Acre
3,1970-05-05 00:00:00,83010,25.0,27.0,32.6,NaN,92.0,Brasiléia,Acre
4,1970-05-05 12:00:00,83010,25.9,24.8,NaN,23.0,91.0,Brasiléia,Acre


![Mapa da Amazônia Legal](https://static.todamateria.com.br/upload/am/az/amazonialegal.jpg)





A Área denominada de Amazônia Legal abrange territórios dos Estados do Acre, Amazonas, Roraima, Rondônia, Mato Grosso, Pará, Tocantins, Amapá e Maranhão. Dessa forma, iremos filtrar os dados obtidos para observar apenas medições de temperatura nesses Estados.

In [0]:
inclusion = ['Acre','Amazonas','Roraima','Rondônia','Mato Grosso','Pará','Tocantins','Amapá','Maranhão']
temp_data = temp_data[temp_data.state.isin(inclusion)]
temp_data

,date,id,tair,tw,tmax,tmin,urmax,name,state
0,1970-05-04 00:00:00,83010,NaN,NaN,32.6,NaN,NaN,Brasiléia,Acre
1,1970-05-04 12:00:00,83010,25.3,24.0,NaN,23.7,90.0,Brasiléia,Acre
2,1970-05-04 18:00:00,83010,29.2,27.1,NaN,NaN,85.0,Brasiléia,Acre
3,1970-05-05 00:00:00,83010,25.0,27.0,32.6,NaN,92.0,Brasiléia,Acre
4,1970-05-05 12:00:00,83010,25.9,24.8,NaN,23.0,91.0,Brasiléia,Acre
...,...,...,...,...,...,...,...,...,...
4339046,2018-12-30 12:00:00,82376,27.5,25.2,NaN,23.4,81.0,Zé Doca,Maranhão
4339047,2018-12-30 18:00:00,82376,32.1,26.0,NaN,NaN,60.0,Zé Doca,Maranhão
4339048,2018-12-31 00:00:00,82376,27.2,25.0,30.2,NaN,83.0,Zé Doca,Maranhão
4339049,2018-12-31 12:00:00,82376,27.7,24.8,NaN,23.4,78.0,Zé Doca,Maranhão


In [0]:
temp_data.isna().sum()

date           0
id             0
tair      107616
tw        183537
tmax     1145842
tmin     1143073
urmax     299221
name           0
state          0
dtype: int64